In [1]:
import smart_wsi_scanner as sws

In [2]:
from smart_wsi_scanner.smartpath import init_pycromanager
core, studio = init_pycromanager()

In [3]:
from smart_wsi_scanner.smartpath import init_pycromanager
core, studio = init_pycromanager()
if core.get_property("MicroPublisher6","Color")=="OFF":
    core.set_property("MicroPublisher6","Color","ON")
if core.get_property("MicroPublisher6","Color - Algorithm CFA") != "G-R-B-G":
    core.set_property("MicroPublisher6","Color - Algorithm CFA", "G-R-B-G")


In [5]:
brushless = "KBD101_Thor_Rotation"
core.get_position(brushless)

296

In [7]:

angle = -10
def ppm_to_thor(angle):
    return (-2*angle + 276)

newAngle = ppm_to_thor(angle)
core.set_position(brushless, newAngle)
core.get_position(brushless)

New angle calculated 296...


296

In [10]:
import smart_wsi_scanner.smartpath as sp
x,y = sp.snap(core)

Capture Failed: SP doesn't recognize : tags['Core-Camera']='MicroPublisher6'


In [27]:
import csv
from dataclasses import dataclass
from typing import List

@dataclass
class ColorPreset:
    name: str
    color: str
    color_algorithm_cfa: str
    color_blue_scale: float
    color_green_scale: float
    color_red_scale: float
    exposure: float
    objective_label: str
    
    def apply_to_camera(self, core, device="MicroPublisher6", objective_device="DObjective"):
            # Apply color and algorithm
            core.set_property(device, "Color", self.color)
            core.set_property(device, "Color - Algorithm CFA", self.color_algorithm_cfa)
            core.set_property(device, "Color - Blue scale", self.color_blue_scale)
            core.set_property(device, "Color - Green scale", self.color_green_scale)
            core.set_property(device, "Color - Red scale", self.color_red_scale)
            core.set_property(device, "Exposure", self.exposure)
            core.set_property(objective_device, "Label", self.objective_label)

def parse_all_presets(cfg_path: str) -> List[ColorPreset]:
    presets = []
    current_name = None
    current_values = {}
    with open(cfg_path, newline='') as csvfile:
        for line in csvfile:
            line = line.strip()
            if not line:
                continue
            # Detect preset header
            if line.startswith("#") or (line.startswith("[") and line.endswith("]")):
                # Save previous preset if exists
                if current_name and current_values:
                    try:
                        presets.append(ColorPreset(name=current_name, **current_values))
                    except Exception as e:
                        print(f"Skipping preset {current_name} due to error: {e}")
                # Start new preset
                current_name = line.strip("#[] ").strip()
                current_values = {}
                continue
            # Parse CSV line
            row = next(csv.reader([line]))
            if len(row) < 6:
                continue
            prop = row[4].lower().replace(" ", "_").replace("-", "_")
            value = row[5]
            # Map config property to dataclass field
            if prop == "color":
                current_values["color"] = value
            elif prop == "color___algorithm_cfa":
                current_values["color_algorithm_cfa"] = value
            elif prop == "color___blue_scale":
                current_values["color_blue_scale"] = float(value)
            elif prop == "color___green_scale":
                current_values["color_green_scale"] = float(value)
            elif prop == "color___red_scale":
                current_values["color_red_scale"] = float(value)
            elif prop == "exposure":
                current_values["exposure"] = float(value)
            elif prop == "label":
                current_values["objective_label"] = value
        # Save last preset
        if current_name and current_values:
            try:
                presets.append(ColorPreset(name=current_name, **current_values))
            except Exception as e:
                print(f"Skipping preset {current_name} due to error: {e}")
    return presets


In [28]:
# Usage
presets = parse_all_presets("../smart_wsi_scanner/configurations/config_PPM.cfg")
#for preset in presets:
#    print(preset)
presets

Skipping preset Preset: Startup due to error: ColorPreset.__init__() missing 5 required positional arguments: 'color_algorithm_cfa', 'color_blue_scale', 'color_green_scale', 'color_red_scale', and 'exposure'
Skipping preset Preset: 10x 0.3 due to error: ColorPreset.__init__() missing 6 required positional arguments: 'color', 'color_algorithm_cfa', 'color_blue_scale', 'color_green_scale', 'color_red_scale', and 'exposure'
Skipping preset Preset: 40x 0.5 red Pol due to error: ColorPreset.__init__() missing 6 required positional arguments: 'color', 'color_algorithm_cfa', 'color_blue_scale', 'color_green_scale', 'color_red_scale', and 'exposure'
Skipping preset Preset: 20x 0.5 due to error: ColorPreset.__init__() missing 6 required positional arguments: 'color', 'color_algorithm_cfa', 'color_blue_scale', 'color_green_scale', 'color_red_scale', and 'exposure'
Skipping preset Preset: 40x 0.75 due to error: ColorPreset.__init__() missing 6 required positional arguments: 'color', 'color_algori

[ColorPreset(name='Preset: ColorOn 10x 5deg', color='ON', color_algorithm_cfa='G-B-R-G', color_blue_scale=2.4344, color_green_scale=1.0, color_red_scale=1.1621, exposure=200.0, objective_label='10x 0.3'),
 ColorPreset(name='Preset: ColorOn 20x 0deg', color='ON', color_algorithm_cfa='G-B-R-G', color_blue_scale=2.2428, color_green_scale=1.014, color_red_scale=1.0, exposure=200.0, objective_label='20x 0.5'),
 ColorPreset(name='Preset: ColorOn 10x 0deg', color='ON', color_algorithm_cfa='G-R-B-G', color_blue_scale=1.0, color_green_scale=1.1691, color_red_scale=1.8056, exposure=500.0, objective_label='10x 0.3'),
 ColorPreset(name='Preset: ColorOn 40x 5deg', color='ON', color_algorithm_cfa='G-B-R-G', color_blue_scale=2.8488, color_green_scale=1.2352, color_red_scale=1.0, exposure=500.0, objective_label='40x 0.75'),
 ColorPreset(name='Preset: ColorOn 10x 50deg', color='ON', color_algorithm_cfa='G-B-R-G', color_blue_scale=3.7246, color_green_scale=1.0, color_red_scale=1.0712, exposure=4.0, obje

In [29]:
for kx,k in enumerate(presets):
    print(kx, k.name)

0 Preset: ColorOn 10x 5deg
1 Preset: ColorOn 20x 0deg
2 Preset: ColorOn 10x 0deg
3 Preset: ColorOn 40x 5deg
4 Preset: ColorOn 10x 50deg
5 Preset: ColorOn 40x 50deg
6 Preset: ColorOn 40x 0deg
7 Preset: ColorOn 20x 50deg
8 Preset: Raw
9 Preset: ColorOn 20x 5deg


In [31]:
presets[1].apply_to_camera(core)

In [ ]:
""

In [4]:
core.get_position()

-55.1

In [3]:
core.get_position("KBD101_Thor_Rotation")
stage_name = "KBD101_28253598"

Exception: java.lang.Exception: No device with label "KBD101_Thor_Rotation"
mmcorej.MMCoreJJNI.CMMCore_getPosition__SWIG_0(Native Method)
mmcorej.CMMCore.getPosition(CMMCore.java:1236)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
org.micromanager.pyjavaz.ZMQServer.runMethod(ZMQServer.java:431)
org.micromanager.pyjavaz.ZMQServer.parseAndExecuteCommand(ZMQServer.java:511)
org.micromanager.pyjavaz.ZMQServer.lambda$initialize$2(ZMQServer.java:121)
java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
java.util.concurrent.FutureTask.run(FutureTask.java:266)
java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
java.lang.Thread.run(Thread.java:74

In [ ]:
brushless = "KBD101_Thor_Rotation";
print("Homing " + brushless + "...");
mmc.home(brushless);
mmc.waitForDevice(brushless);
// One more time, in case it failed the first time.
mmc.home(brushless);
mmc.waitForDevice(brushless);
print("Motor homed at: " + mmc.getPosition(brushless));

In [12]:
import platform
import psutil

if platform.system() != "Windows":
    print(platform)

for proc in psutil.process_iter(['name']):
    try:
        if "ImageJ.exe" in proc.info['name'] or "Micro-Manager" in proc.info['name']:
            print("runing")
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass


not running


In [21]:
for proc in psutil.process_iter(['name']):
    name = proc.info['name']
    if name.startswith('j'):print(name); break
    

javaw.exe


True